<a href="https://colab.research.google.com/github/shubhangickore/Python_practice/blob/main/pr2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
print("hello world")

hello world
